# LSST Twilight Planner (Modular) — Notebook

This notebook demonstrates the use of the `twilight_planner_pkg` package to
generate twilight observing plans. The workflow is:

1. Set up imports and paths for the input SN catalog and output directory.
2. Build a fully specified `PlannerConfig` object with all optional parameters
   shown (defaults are used unless otherwise noted).
3. Run the planner and inspect the CSV outputs written to `OUTDIR`.


In [4]:
# --- Imports and path setup ---
import sys
import os

# Ensure local package is importable
sys.path.append("./")

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# Paths: adjust if your data/output lives elsewhere
CSV_PATH = "../data/ATLAS_2021_to25_cleaned.csv"
OUTDIR = "../twilight_outputs"
os.makedirs(OUTDIR, exist_ok=True)

print("CSV exists:", os.path.exists(CSV_PATH))
print("Output dir:", OUTDIR)

CSV exists: True
Output dir: ../twilight_outputs


## Configure the planner

The next cell defines a complete `PlannerConfig`. Every parameter of the
dataclass is included so the notebook doubles as documentation. Edit any value
to explore different strategies or hardware assumptions.


In [5]:
# ---- User-editable parameters ----

# Date range (UTC)
START_DATE = "2024-01-01"
END_DATE = "2024-01-03"

# -- Site --
LAT_DEG = -30.2446
LON_DEG = -70.7494
HEIGHT_M = 2663

# -- Visibility --
MIN_ALT_DEG = 20.0
TWILIGHT_SUN_ALT_MIN_DEG = -15.0  # minimum altitude for twilight
TWILIGHT_SUN_ALT_MAX_DEG = -5.0  # maximum altitude for twilight


# -- Filters and hardware --
FILTERS = ["g", "r", "i", "z"]
CAROUSEL_CAPACITY = 5
FILTER_CHANGE_S = 120.0
READOUT_S = 2.0
FILTER_CHANGE_TIME_S = None  # legacy alias
READOUT_TIME_S = None  # legacy alias
INTER_EXPOSURE_MIN_S = 15.0
EXPOSURE_BY_FILTER = {"g": 5.0, "r": 5.0, "i": 5.0, "z": 5.0}
# NEW — cadence knobs
CADENCE_ENABLE = True
CADENCE_PER_FILTER = True
CADENCE_DAYS_TARGET = 3.0
CADENCE_JITTER_DAYS = 0.25
CADENCE_DAYS_TOLERANCE = 0.5
CADENCE_BONUS_SIGMA_DAYS = 0.5
CADENCE_BONUS_WEIGHT = 0.25

# Allow colors in one visit (if you want them)
MAX_FILTERS_PER_VISIT = 3  # was 1
ALLOW_FILTER_CHANGES_IN_TWILIGHT = True  # was False
START_FILTER = FILTERS[0]
SUN_ALT_POLICY = [
    (-18.0, -15.0, ["y", "z", "i"]),
    (-15.0, -12.0, ["z", "i", "r"]),
    (-12.0, 0.0, ["i", "z", "y"]),
]

# -- Slew model --
SLEW_SMALL_DEG = 3.5
SLEW_SMALL_TIME_S = 4.0
SLEW_RATE_DEG_PER_S = 5.25
SLEW_SETTLE_S = 1.0

# -- Moon --
MIN_MOON_SEP_BY_FILTER = {
    "u": 80.0,
    "g": 50.0,
    "r": 35.0,
    "i": 30.0,
    "z": 25.0,
    "y": 20.0,
}
REQUIRE_SINGLE_TIME_FOR_ALL = True

# -- Time caps --
PER_SN_CAP_S = 120.0
MORNING_CAP_S = 600.0
EVENING_CAP_S = 600.0
TWILIGHT_STEP_MIN = 2
MAX_SN_PER_NIGHT = 10

# -- Priority strategy --
PRIORITY_STRATEGY = "hybrid"  # or "lc"
HYBRID_DETECTIONS = 2
HYBRID_EXPOSURE = 300.0
LC_DETECTIONS = 5
LC_EXPOSURE = 300.0

# -- Photometry / sky --
PIXEL_SCALE_ARCSEC = 0.2
ZPT1S = None
K_M = None
FWHM_EFF = None
READ_NOISE_E = 6.0
GAIN_E_PER_ADU = 1.0
ZPT_ERR_MAG = 0.01
DARK_SKY_MAG = None
TWILIGHT_DELTA_MAG = 2.5  # fallback if rubin_sim.skybrightness unavailable

# -- SIMLIB output --
SIMLIB_OUT = None  # e.g. "twilight.simlib"
SIMLIB_SURVEY = "LSST"
SIMLIB_FILTERS = "grizy"
SIMLIB_PIXSIZE = 0.2
SIMLIB_NPE_PIXEL_SATURATE = 1.0e6
SIMLIB_PHOTFLAG_SATURATE = 4096
SIMLIB_PSF_UNIT = "arcsec"

# -- Miscellaneous --
TYPICAL_DAYS_BY_TYPE = {
    "Ia": 70,
    "II-P": 100,
    "II-L": 70,
    "IIn": 120,
    "IIb": 70,
    "Ib": 60,
    "Ic": 60,
}
DEFAULT_TYPICAL_DAYS = 60
RA_COL = None
DEC_COL = None
DISC_COL = None
NAME_COL = None
TYPE_COL = None

# Assemble the configuration object with all parameters exposed
cfg = PlannerConfig(
    lat_deg=LAT_DEG,
    lon_deg=LON_DEG,
    height_m=HEIGHT_M,
    min_alt_deg=MIN_ALT_DEG,
    twilight_sun_alt_min_deg=TWILIGHT_SUN_ALT_MIN_DEG,
    twilight_sun_alt_max_deg=TWILIGHT_SUN_ALT_MAX_DEG,
    filters=FILTERS,
    carousel_capacity=CAROUSEL_CAPACITY,
    filter_change_s=FILTER_CHANGE_S,
    readout_s=READOUT_S,
    filter_change_time_s=FILTER_CHANGE_TIME_S,
    readout_time_s=READOUT_TIME_S,
    inter_exposure_min_s=INTER_EXPOSURE_MIN_S,
    exposure_by_filter=EXPOSURE_BY_FILTER,
    max_filters_per_visit=MAX_FILTERS_PER_VISIT,
    start_filter=START_FILTER,
    sun_alt_policy=SUN_ALT_POLICY,
    slew_small_deg=SLEW_SMALL_DEG,
    slew_small_time_s=SLEW_SMALL_TIME_S,
    slew_rate_deg_per_s=SLEW_RATE_DEG_PER_S,
    slew_settle_s=SLEW_SETTLE_S,
    min_moon_sep_by_filter=MIN_MOON_SEP_BY_FILTER,
    require_single_time_for_all_filters=REQUIRE_SINGLE_TIME_FOR_ALL,
    per_sn_cap_s=PER_SN_CAP_S,
    morning_cap_s=MORNING_CAP_S,
    evening_cap_s=EVENING_CAP_S,
    twilight_step_min=TWILIGHT_STEP_MIN,
    max_sn_per_night=MAX_SN_PER_NIGHT,
    cadence_enable=CADENCE_ENABLE,
    cadence_per_filter=CADENCE_PER_FILTER,
    cadence_days_target=CADENCE_DAYS_TARGET,
    cadence_jitter_days=CADENCE_JITTER_DAYS,
    cadence_days_tolerance=CADENCE_DAYS_TOLERANCE,
    cadence_bonus_sigma_days=CADENCE_BONUS_SIGMA_DAYS,
    cadence_bonus_weight=CADENCE_BONUS_WEIGHT,
    hybrid_detections=HYBRID_DETECTIONS,
    hybrid_exposure_s=HYBRID_EXPOSURE,
    lc_detections=LC_DETECTIONS,
    lc_exposure_s=LC_EXPOSURE,
    priority_strategy=PRIORITY_STRATEGY,
    pixel_scale_arcsec=PIXEL_SCALE_ARCSEC,
    zpt1s=ZPT1S,
    k_m=K_M,
    fwhm_eff=FWHM_EFF,
    read_noise_e=READ_NOISE_E,
    gain_e_per_adu=GAIN_E_PER_ADU,
    zpt_err_mag=ZPT_ERR_MAG,
    dark_sky_mag=DARK_SKY_MAG,
    twilight_delta_mag=TWILIGHT_DELTA_MAG,
    simlib_out=SIMLIB_OUT,
    simlib_survey=SIMLIB_SURVEY,
    simlib_filters=SIMLIB_FILTERS,
    simlib_pixsize=SIMLIB_PIXSIZE,
    simlib_npe_pixel_saturate=SIMLIB_NPE_PIXEL_SATURATE,
    simlib_photflag_saturate=SIMLIB_PHOTFLAG_SATURATE,
    simlib_psf_unit=SIMLIB_PSF_UNIT,
    typical_days_by_type=TYPICAL_DAYS_BY_TYPE,
    default_typical_days=DEFAULT_TYPICAL_DAYS,
    ra_col=RA_COL,
    dec_col=DEC_COL,
    disc_col=DISC_COL,
    name_col=NAME_COL,
    type_col=TYPE_COL,
    allow_filter_changes_in_twilight=ALLOW_FILTER_CHANGES_IN_TWILIGHT,
)

cfg

PlannerConfig(lat_deg=-30.2446, lon_deg=-70.7494, height_m=2663, min_alt_deg=20.0, typical_days_by_type={}, default_typical_days=30, slew_small_deg=3.0, slew_small_time_s=2.0, slew_rate_deg_per_s=3.5, slew_settle_s=2.0, readout_s=2.0, filter_change_s=120.0, filters=['g', 'r', 'i', 'z'], exposure_by_filter={'g': 5.0, 'r': 5.0, 'i': 5.0, 'z': 5.0}, carousel_capacity=6, twilight_step_min=2, evening_cap_s=600.0, morning_cap_s=600.0, max_sn_per_night=10, per_sn_cap_s=120.0, min_moon_sep_by_filter={'g': 30.0, 'r': 25.0, 'i': 20.0, 'z': 15.0}, require_single_time_for_all_filters=True, priority_strategy='hybrid', hybrid_detections=2, hybrid_exposure_s=300.0, lc_detections=5, lc_exposure_s=300.0, lc_phase_range=(-7.0, 20.0), ra_col=None, dec_col=None, disc_col=None, name_col=None, type_col=None)

## Run the planner

`plan_twilight_range_with_caps` is invoked below.  It reads the SN catalog from
`CSV_PATH`, writes output CSVs to `OUTDIR`, and prints progress information.
Two files are produced for each run:

- `lsst_twilight_plan_*.csv` — per-SN schedule
- `lsst_twilight_summary_*.csv` — per-night summary


In [6]:
# Execute the planner and return data frames
perSN_df, nights_df = plan_twilight_range_with_caps(
    csv_path=CSV_PATH,
    outdir=OUTDIR,
    start_date=START_DATE,
    end_date=END_DATE,
    cfg=cfg,
    verbose=True,
)

print("Per-SN rows:", len(perSN_df), "  Nights summary rows:", len(nights_df))
perSN_df.head(10)

RA raw (numeric) range:  min=0.043167, max=359.855458
Dec raw (numeric) range: min=-87.141200, max=87.960305


Nights:   0%|          | 0/3 [00:00<?, ?night/s]

2024-01-01: eligible=81 visible=62 planned_total=9 evening_twilight=2024-01-01T23:44:00+00:00 morning_twilight=2024-01-02T08:09:00+00:00 filters=g,r,i,z
2024-01-02: eligible=85 visible=71 planned_total=7 evening_twilight=2024-01-02T23:45:00+00:00 morning_twilight=2024-01-03T08:10:00+00:00 filters=g,r,i,z
2024-01-03: eligible=86 visible=74 planned_total=7 evening_twilight=2024-01-03T23:45:00+00:00 morning_twilight=2024-01-04T08:11:00+00:00 filters=g,r,i,z
Wrote:
  ../twilight_outputs/lsst_twilight_plan_2024-01-01_to_2024-01-03.csv
  ../twilight_outputs/lsst_twilight_summary_2024-01-01_to_2024-01-03.csv
Rows: per-SN=23, nights*windows=9
Per-SN rows: 23   Nights summary rows: 9


,date,twilight_window,SN,RA_deg,Dec_deg,best_twilight_time_utc,best_alt_deg,priority_score,filters,exposure_s,readout_s,filter_changes_s,slew_s,total_time_s
0,2024-01-01,morning,2023yxi,47.417777,-31.222880,2024-01-01T01:12:00+00:00,89.09,1.0,g,5.0,2.0,0.0,0.0,7.0
1,2024-01-01,morning,2023yqc,38.192499,-38.310448,2024-01-01T00:36:00+00:00,82.04,1.0,g,5.0,2.0,0.0,6.1,13.1
2,2024-01-01,morning,2023zzn,46.861701,-42.658672,2024-01-01T01:10:00+00:00,77.67,1.0,g,5.0,2.0,0.0,5.4,12.4
3,2024-01-01,morning,2023ypx,18.428125,-41.627731,2023-12-31T23:44:00+00:00,77.51,1.0,g,5.0,2.0,0.0,9.1,16.1
4,2024-01-01,morning,2023ywq,15.922583,-39.874981,2023-12-31T23:44:00+00:00,77.89,1.0,g,5.0,2.0,0.0,4.0,11.0
5,2024-01-01,morning,2023zrc,17.044547,-36.059907,2023-12-31T23:44:00+00:00,81.17,1.0,g,5.0,2.0,0.0,4.3,11.3
6,2024-01-01,evening,2024C,145.975500,-29.727481,2024-01-01T08:09:00+00:00,85.01,1.0,g,5.0,2.0,0.0,0.0,7.0
7,2024-01-01,evening,2023zce,141.587417,-31.226239,2024-01-01T08:09:00+00:00,81.23,1.0,g,5.0,2.0,0.0,4.3,11.3
8,2024-01-01,evening,2023zea,148.281780,-37.998441,2024-01-01T08:09:00+00:00,81.64,1.0,g,5.0,2.0,0.0,5.6,12.6
9,2024-01-02,morning,2023yqc,38.192499,-38.310448,2024-01-02T00:32:00+00:00,82.04,1.0,g,5.0,2.0,0.0,0.0,7.0


In [7]:
# Show the nightly summary
nights_df

,date,twilight_window,n_candidates,n_planned,sum_time_s,window_cap_s
0,2024-01-01,morning,6,6,70.9,600
1,2024-01-01,evening,3,3,30.9,600
2,2024-01-01,W2,1,0,0.0,0
3,2024-01-02,morning,3,3,33.3,600
4,2024-01-02,evening,4,4,42.4,600
5,2024-01-02,W2,3,0,0.0,0
6,2024-01-03,morning,3,3,37.3,600
7,2024-01-03,evening,4,4,42.4,600
8,2024-01-03,W2,3,0,0.0,0



## Inspect outputs on disk


In [8]:
import glob
import os

files = sorted(glob.glob(os.path.join(OUTDIR, "lsst_twilight_*.csv")))
files

['../twilight_outputs/lsst_twilight_plan_2024-01-01_to_2024-01-03.csv',
 '../twilight_outputs/lsst_twilight_summary_2024-01-01_to_2024-01-03.csv']